In [ ]:
# Dependencias
!pip install pypdf langchain langchain-community langchain-core langchain-openai chromadb python-dotenv

# Usar solo en Databricks
# dbutils.library.restartPython()

# Ejercicio de Ingeniería de Software con Inteligencia Artificial

## Descripción de la Prueba

El objetivo de esta prueba es evaluar tus habilidades y conocimientos en el desarrollo de software con inteligencia artificial, específicamente en sistemas RAG (Retrieval-Augmented Generation), embeddings, y bases de datos vectoriales.

Se te proporcionarán tres archivos PDF con reportes de resultados de Grupo Bimbo, así como las llaves de API y los endpoints de OpenAI. Tu tarea será utilizar estos recursos para desarrollar un sistema que pueda realizar tareas de retrieval de manera eficiente.

## Objetivos de la Prueba

1. Comprensión de los datos: Deberás ser capaz de procesar y entender los datos proporcionados en los archivos PDF.

2. Uso de OpenAI y embeddings: Deberás demostrar tu habilidad para trabajar con OpenAI y embeddings para realizar tareas de retrieval.

3. Implementación de bases de datos vectoriales: Deberás implementar una base de datos vectorial para manejar eficientemente los datos.

4. Desarrollo de un sistema RAG: Finalmente, deberás demostrar tu capacidad para desarrollar un sistema RAG que pueda realizar tareas de retrieval y generación de texto de manera eficiente.

## Presentación del Código

Tu código debe seguir las mejores prácticas de codificación y puede presentarse en notebooks de Jupyter. Esto incluye:

· Legibilidad: El código debe ser fácil de leer y entender.

· Comentarios: Debes incluir comentarios que expliquen tu razonamiento y las decisiones de diseño que tomaste.

· Estructura: El código debe estar bien estructurado y organizado.

· Pruebas: Debes incluir pruebas para asegurar que tu código funciona como se espera.

Para revisión de este proyecto ser realizará una sesión en línea donde presentarás tu abordaje el problema, tu resolución técnica y código generado.

- APIkey: 75433f0e2ce040ebb90a2fa457fbc815
- EndPoint: https://ennovenoai.openai.azure.com/
- Modelo Openai: EnnovenGPTTurbo
- Modelo Ada-002: EnnovenAda

Anexo igualmente los PDFs que sirven como fuente de datos para el sistema RAG, recordando que buscamos una búsqueda enriquecida: (Datos + metadata)

El archivo contiene tablas, e imágenes, así como textos. Se espera del ejercicio el procesamiento de las tablas e imágenes dentro del modelo. Así como una aproximación de reranking de los resultados para un mejor contexto de respuesta.

Las API Keys serán cambiadas una ves terminado el ejercicio.

In [1]:
# Bibliotecas para interactuar con el LLM
from langchain_openai import AzureChatOpenAI
# Para generar los embeddings
from langchain_openai import AzureOpenAIEmbeddings

# Para cargar los datos en formato PDF
from langchain_community.document_loaders import PyPDFLoader
# Para el splitting de los textos
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Para almacenar los embeddings en ChromaDB
from langchain_community.vectorstores import Chroma
# Para generar el RAG usando la base de datos vectorial
from langchain.chains import RetrievalQAWithSourcesChain

# Interactuar con el sistema de archivos
import os
import glob
from dotenv import load_dotenv

# Cargar las variables de ambiente desde el .env
load_dotenv()

True

## Credenciales de accesso

Para Databricks: Las credenciales de acceso al modelo han sido almacenadas en variables de ambiente en la configuración del Clúster para mayor seguridad.

<img src="./img/databricks-env-variables.png">

Para entornos de prueba local: Las credenciales se encuentran en un archivo `.env`

<img src="./img/variables-de-ambiente.png">

In [2]:
openai_api_version = os.environ["OPENAI_API_VERSION"]
azure_deployment = os.environ["AZURE_DEPLOYMENT"]
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
model_name = os.environ["MODEL_NAME"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

Creamos una instancia para usar el modelo de ChatGPT de Azure y el modelo para generar los embeddings

In [3]:
# Crear una conexión con el modelo GPT de Azure
model = AzureChatOpenAI(
    api_key = azure_openai_api_key, # API Key
    azure_endpoint = azure_openai_endpoint, # Endpoint
    openai_api_version = openai_api_version, # Obtenido de la documentación
    model_name = model_name, # Nombre del modelo
    temperature = 0 # Temperatura del modelo GPT
)

In [4]:
# Definir el modelo para generar los embeddings
embedding_model = AzureOpenAIEmbeddings(
    api_key = azure_openai_api_key, # API Key
    azure_endpoint = azure_openai_endpoint, # Endpoint
    azure_deployment = azure_deployment, # Nombre del modelo de embeddings
    openai_api_version = openai_api_version # Obtenido de la documentación
)

# Cargar los datos para el RAG

Las fuentes de datos pueden ser variadas, pero para este ejercicio tenemos archivos en formato PDF dentro de directorio `data`.

Podemos acceder a ese directorio y filtrar los archivos usando expresiones regulares.

In [5]:
# Hiperparametros para el splitting
chunk_size = 1000
chunk_overlap = 200

# Inicializar el splitter con los hiperparametros
splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, 
    chunk_overlap = chunk_overlap,
    separators = ["."]
)

In [6]:


# Inicializar el splitter con los hiperparametros
splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, 
    chunk_overlap = chunk_overlap,
    separators = ["."]
)

# Inicializar la base de datos vectorial
vectordb = Chroma(
    persist_directory = "db",
    embedding_function = embedding_model,
    collection_name = "ennoven_rag"
)

# Configurar Chroma para hacer datos persistentes en disco
vectordb.persist()

# Para cada PDF dentro del directorio
# for file in os.listdir("data"):
for file in glob.glob("data/*.pdf"):
# Crear una conexión con el PDF
    print(f"Processing file: {file}")
    loader = PyPDFLoader(f"{file}")
    
    # Extraer el texto del PDF
    print(f"Extracting text from file: {file}")
    pdf_data = loader.load()
    
    # Dividir el contenido del PDF en documentos
    print(f"Splitting docs for {file}")
    docs = splitter.split_documents(pdf_data)
    
    # Crear los embeddings de los documentos usando el modelo y almacenarlos en ChromaDB
    print(f"Creating embeddings for docs in {file}")
    docstorage = Chroma.from_documents(docs, embedding_model)

    print("Embeddings created", end="\n\n")

Processing file: data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Extracting text from file: data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Splitting docs for data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Creating embeddings for docs in data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Embeddings created

Processing file: data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Extracting text from file: data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Splitting docs for data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Creating embeddings for docs in data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf


Una vez almacenados los embeddings en la base de datos vectorias, podemos usarla para construir un RAG (básico) para obtener en base al contexto de los archivos PDF que extraímos.

Usamos la función `RetrievalQAWithSourcesChain` para obtener una respuesta junto con su metadata, que incluye la fuente y la similitud. Especialmente útil para identificar si la respuesta generada por el modelo ha sido una alucinación.

In [ ]:
# Retornar las referencias al RAG desde la base de datos vectorial de ChormaDB
print(f"Returning references to RAG data sources")
rag = RetrievalQAWithSourcesChain.from_chain_type(
    llm = model, # Usar el modelo GPT
    chain_type = "stuff",
    retriever = docstorage.as_retriever() # Usar el contenido de la base de datos
)

Para probar este modelo primitivo, podemos usar 

In [ ]:
query = "Cuales son los principales prestamistas bancarios de la compañia, con la que se mantiene un endeudamiento"
rag.invoke(query)

In [ ]:
query = "Desglosa la deuda con los principales prestamistas bancarios de la compañia"
rag.invoke(query)